In [1]:
from utils import load_spam_dataset
import pandas as pd

# Load Data

In [2]:
df_train, df_test = load_spam_dataset(load_train_labels=True)

# Writing Slicing Functions (SFs)

In [4]:
#Execute slicing functions
from SpamSlicingFunctions import *
sfs = [short_comment]

In [5]:
#Visualize
from snorkel.slicing import slice_dataframe
short_comment_df = slice_dataframe(df_test, short_comment)
cols = ['text', 'label']
short_comment_df[cols].head()

# Monitor Slice Performance

In [9]:
#Train classifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from utils import df_to_features

vectorizer = CountVectorizer(ngram_range=(1,1))
X_train, Y_train = df_to_features(vectorizer, df_train, "train")
X_test, Y_test = df_to_features(vectorizer, df_test, "test")

sklearn_model = LogisticRegression(C=0.001, solver="liblinear")
sklearn_model.fit(X=X_train, y=Y_train)

LogisticRegression(C=0.001, solver='liblinear')

In [10]:
#Evaluate overall performance
from sklearn.metrics import f1_score
from snorkel.utils import preds_to_probs
preds_test = sklearn_model.predict(X_test)
probs_test = preds_to_probs(preds_test, 2)

print(f"Test set F1: {100*f1_score(Y_test, preds_test):.1f}%")

Test set F1: 92.5%


In [11]:
#Evaluate overall and slice performance
from snorkel.analysis import Scorer
from snorkel.slicing import PandasSFApplier

applier = PandasSFApplier(sfs)
S_test = applier.apply(df_test)
scorer = Scorer(metrics=["f1"])
scorer.score_slices(S=S_test, 
                    golds=Y_test, 
                    preds=preds_test, 
                    probs=probs_test, 
                    as_dataframe=True)

100%|████████████████████████████████████████| 250/250 [00:00<00:00, 28448.30it/s]


,f1
overall,0.925000
short_comment,0.666667


# Additional SFs